### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

Student ID       student_name gender grade        school_name  \
0           0       Paul Bradley      M   9th  Huang High School   
1           1       Victor Smith      M  12th  Huang High School   
2           2    Kevin Rodriguez      M  12th  Huang High School   
3           3  Dr. Richard Scott      M  12th  Huang High School   
4           4         Bonnie Ray      F   9th  Huang High School   

   reading_score  math_score  School ID      type  size   budget  
0             66          79          0  District  2917  1910635  
1             94          61          0  District  2917  1910635  
2             90          60          0  District  2917  1910635  
3             67          58          0  District  2917  1910635  
4             97          84          0  District  2917  1910635

## District Summary

* Calculate the total number of schools
* Calculate the total number of students
* Calculate the total budget
* Calculate the average math score 
* Calculate the average reading score
* Calculate the percentage of students with a passing math score (70 or greater)
* Calculate the percentage of students with a passing reading score (70 or greater)
* Calculate the percentage of students who passed math **and** reading (% Overall Passing)
* Create a dataframe to hold the above results
* Optional: give the displayed data cleaner formatting

In [50]:
grp = school_data_complete.groupby('school_name')

total_students = school_data_complete.student_name.count()
passing_math_count = school_data_complete.loc[school_data_complete["math_score"] >= 70]["math_score"].count()
passing_reading_count = school_data_complete.loc[school_data_complete["reading_score"] >= 70]["reading_score"].count()
passing_overall_count = school_data_complete.loc[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)]["Student ID"].count()

district_summary = pd.DataFrame({
    'Total Schools': [school_data_complete.school_name.nunique()],
    'Total Students': [total_students],
    'Total Budget': [grp.budget.max().sum()],
    'Average Math Score': [school_data_complete["math_score"].sum()/total_students],
    'Average Reading Score': [school_data_complete["reading_score"].sum()/total_students],
    '% Passing Math': [(passing_math_count/total_students)*100],
    '% Passing Reading': [(passing_reading_count/total_students)*100],
    '% Overall Passing': [(passing_overall_count/total_students)*100]
    
})


district_summary.style.format({"Total Students": "{:,}",
                               "Total Budget": "${:,.2f}", 
                               "Average Reading Score": "{:.2f}", 
                               "Average Math Score": "{:.2f}", 
                               "% Passing Math": "{:.2f}%", 
                               "% Passing Reading": "{:.2f}%", 
                               "% Overall Passing": "{:.2f}%" 
})

## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [108]:
school_summary = school_data_complete.groupby("school_name")
school_types = school_data.set_index("school_name")["type"]
school_total_students = school_summary["Student ID"].count()
school_budget = school_data.set_index("school_name")["budget"]
student_budget = school_budget / school_total_students
school_math_average = school_summary["math_score"].mean()
school_reading_average = school_summary["reading_score"].mean()
school_passing_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby("school_name")["Student ID"].count()/school_total_students*100
school_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("school_name")["Student ID"].count()/school_total_students*100
school_passing_overall = school_data_complete[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)].groupby("school_name")["Student ID"].count()/school_total_students*100

school_summary = pd.DataFrame({'School Type': school_types,
                               'Total Students': school_total_students,
                               'Total Budget': school_budget,
                               'Per Student Budget': student_budget,
                               'Average Math Score': school_math_average,
                               'Average Reading Score': school_reading_average,
                               '% Passing Math': school_passing_math,
                               '% Passing Reading': school_passing_reading,
                               '% Overall Passing': school_passing_overall
                              })


school_summary.style.format({"Total Students": "{:,}",
                               "Total Budget": "${:,.2f}", 
                               "Per Student Budget": "${:.2f}",
                               "Average Math Score": "{:.2f}", 
                               "Average Reading Score": "{:.2f}", 
                               "% Passing Math": "{:.2f}%", 
                               "% Passing Reading": "{:.2f}%", 
                               "% Overall Passing": "{:.2f}%" 
                              })

## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [104]:
school_summary.sort_values(["% Overall Passing"], ascending = False).head()

School Type  Total Students  Total Budget  \
Cabrera High School     Charter            1858       1081356   
Thomas High School      Charter            1635       1043130   
Griffin High School     Charter            1468        917500   
Wilson High School      Charter            2283       1319574   
Pena High School        Charter             962        585858   

                     Average Math Score  Average Reading Score  \
Cabrera High School           83.061895              83.975780   
Thomas High School            83.418349              83.848930   
Griffin High School           83.351499              83.816757   
Wilson High School            83.274201              83.989488   
Pena High School              83.839917              84.044699   

                     % Passing Math  % Passing Reading  % Overall Passing  
Cabrera High School       94.133477          97.039828          91.334769  
Thomas High School        93.272171          97.308869          90.948012  
Griffin High School       93.392371          97.138965          90.599455  
Wilson High School        93.867718          96.539641          90.582567  
Pena High School          94.594595          95.945946          90.540541

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [103]:
school_summary.sort_values(["% Overall Passing"], ascending = True).head()

School Type  Total Students  Total Budget  \
Rodriguez High School    District            3999       2547363   
Figueroa High School     District            2949       1884411   
Huang High School        District            2917       1910635   
Hernandez High School    District            4635       3022020   
Johnson High School      District            4761       3094650   

                       Average Math Score  Average Reading Score  \
Rodriguez High School           76.842711              80.744686   
Figueroa High School            76.711767              81.158020   
Huang High School               76.629414              81.182722   
Hernandez High School           77.289752              80.934412   
Johnson High School             77.072464              80.966394   

                       % Passing Math  % Passing Reading  % Overall Passing  
Rodriguez High School       66.366592          80.220055          52.988247  
Figueroa High School        65.988471          80.739234          53.204476  
Huang High School           65.683922          81.316421          53.513884  
Hernandez High School       66.752967          80.862999          53.527508  
Johnson High School         66.057551          81.222432          53.539172

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [83]:
math_ninth = school_data_complete.loc[school_data_complete["grade"] == "9th"].groupby("school_name")["math_score"].mean()
math_tenth = school_data_complete.loc[school_data_complete["grade"] == "10th"].groupby("school_name")["math_score"].mean()
math_eleventh = school_data_complete.loc[school_data_complete["grade"] == "11th"].groupby("school_name")["math_score"].mean()
math_twelfth = school_data_complete.loc[school_data_complete["grade"] == "12th"].groupby("school_name")["math_score"].mean()

math_grades = pd.DataFrame({"9th": math_ninth,"10th": math_tenth,"11th": math_eleventh,"12th": math_twelfth})

math_grades

9th       10th       11th       12th
school_name                                                      
Bailey High School     77.083676  76.996772  77.515588  76.492218
Cabrera High School    83.094697  83.154506  82.765560  83.277487
Figueroa High School   76.403037  76.539974  76.884344  77.151369
Ford High School       77.361345  77.672316  76.918058  76.179963
Griffin High School    82.044010  84.229064  83.842105  83.356164
Hernandez High School  77.438495  77.337408  77.136029  77.186567
Holden High School     83.787402  83.429825  85.000000  82.855422
Huang High School      77.027251  75.908735  76.446602  77.225641
Johnson High School    77.187857  76.691117  77.491653  76.863248
Pena High School       83.625455  83.372000  84.328125  84.121547
Rodriguez High School  76.859966  76.612500  76.395626  77.690748
Shelton High School    83.420755  82.917411  83.383495  83.778976
Thomas High School     83.590022  83.087886  83.498795  83.497041
Wilson High School     83.085578  83.724422  83.195326  83.035794
Wright High School     83.264706  84.010288  83.836782  83.644986

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [85]:
reading_ninth = school_data_complete.loc[school_data_complete["grade"] == "9th"].groupby("school_name")["reading_score"].mean()
reading_tenth = school_data_complete.loc[school_data_complete["grade"] == "10th"].groupby("school_name")["reading_score"].mean()
reading_eleventh = school_data_complete.loc[school_data_complete["grade"] == "11th"].groupby("school_name")["reading_score"].mean()
reading_twelfth = school_data_complete.loc[school_data_complete["grade"] == "12th"].groupby("school_name")["reading_score"].mean()

reading_grades = pd.DataFrame({"9th": reading_ninth,"10th": reading_tenth,"11th": reading_eleventh,"12th": reading_twelfth})

reading_grades

9th       10th       11th       12th
school_name                                                      
Bailey High School     81.303155  80.907183  80.945643  80.912451
Cabrera High School    83.676136  84.253219  83.788382  84.287958
Figueroa High School   81.198598  81.408912  80.640339  81.384863
Ford High School       80.632653  81.262712  80.403642  80.662338
Griffin High School    83.369193  83.706897  84.288089  84.013699
Hernandez High School  80.866860  80.660147  81.396140  80.857143
Holden High School     83.677165  83.324561  83.815534  84.698795
Huang High School      81.290284  81.512386  81.417476  80.305983
Johnson High School    81.260714  80.773431  80.616027  81.227564
Pena High School       83.807273  83.612000  84.335938  84.591160
Rodriguez High School  80.993127  80.629808  80.864811  80.376426
Shelton High School    84.122642  83.441964  84.373786  82.781671
Thomas High School     83.728850  84.254157  83.585542  83.831361
Wilson High School     83.939778  84.021452  83.764608  84.317673
Wright High School     83.833333  83.812757  84.156322  84.073171

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [18]:
bins = [0, 584.9, 629.9, 644.9, 679.9, 10000]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]
school_summary["Age Ranges"] = pd.cut(purchase_data["Age"], bins=bins, labels=labels)

Average Math Score  Average Reading Score  \
Spending Ranges (Per Student)                                              
<$585                                       83.46                  83.93   
$585-630                                    81.90                  83.16   
$630-645                                    78.52                  81.62   
$645-680                                    77.00                  81.03   

                               % Passing Math  % Passing Reading  \
Spending Ranges (Per Student)                                      
<$585                                   93.46              96.61   
$585-630                                87.13              92.72   
$630-645                                73.48              84.39   
$645-680                                66.16              81.13   

                               % Overall Passing  
Spending Ranges (Per Student)                     
<$585                                      90.37  
$585-630                                   81.42  
$630-645                                   62.86  
$645-680                                   53.53

In [111]:
bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]
school_data_complete["Spending Ranges (Per Student)"] = pd.cut(school_data_complete["budget"]/school_data_complete["size"], bins=bins, labels=labels)

scores_by_school = school_data_complete.groupby("Spending Ranges (Per Student)")

#working on finishing. 


NameError: name 'school_performance' is not defined

## Scores by School Size

* Perform the same operations as above, based on school size.

Average Math Score  Average Reading Score  % Passing Math  \
School Size                                                                     
Small (<1000)                83.821598              83.929843       93.550225   
Medium (1000-2000)           83.374684              83.864438       93.599695   
Large (2000-5000)            77.746417              81.344493       69.963361   

                    % Passing Reading  % Overall Passing  
School Size                                               
Small (<1000)               96.099437          89.883853  
Medium (1000-2000)          96.790680          90.621535  
Large (2000-5000)           82.766634          58.286003

## Scores by School Type

* Perform the same operations as above, based on school type

Average Math Score  Average Reading Score  % Passing Math  \
School Type                                                              
Charter               83.473852              83.896421       93.620830   
District              76.956733              80.966636       66.548453   

             % Passing Reading  % Overall Passing  
School Type                                        
Charter              96.586489          90.432244  
District             80.799062          53.672208